# Imports

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)

# Datensätze

In [3]:
%store -r df_matches
%store -r df_players
%store -r df_rankings
%store -r df_matches_players

# Hilfsfunktionen

In [4]:
def fillna_0_astype_int64(dataframe:pd.DataFrame, columns:list=["Siege", "Niederlagen"]):

    for column in columns:
        dataframe[column] = dataframe[column].fillna(0).astype("int64")
    
    return dataframe

In [5]:
def siegesquote_funktion(dataframe:pd.DataFrame):

    dataframe["Siegesquote"] = dataframe["Siege"] / (dataframe["Siege"]+ dataframe["Niederlagen"])

    return dataframe

<br>

# Fragestellung 3
### Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?

In [6]:
df_matches_players

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,name_first_winner,name_last_winner,dob_winner,height_winner,wikidata_winner,name_first_loser,name_last_loser,dob_loser,height_loser,wikidata_loser
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,Douglas Smith,U,NaN,AUS,NaN,110196,NaN,NaN,Peter Ledbetter,U,NaN,UNK,24.0,6-1 7-5,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Douglas,Smith,NaN,NaN,NaN,Peter,Ledbetter,19440703.0,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,Louis Pretorius,R,NaN,RSA,NaN,209536,NaN,NaN,Maurice Pollock,U,NaN,IRL,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Louis,Pretorius,19930829.0,NaN,NaN,Maurice,Pollock,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,Cecil Pedlow,U,NaN,IRL,NaN,209535,NaN,NaN,John Mulvey,U,NaN,IRL,NaN,6-2 6-2,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cecil,Pedlow,NaN,NaN,NaN,John,Mulvey,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,Tom Okker,R,178.0,NED,24.3,209534,NaN,NaN,Unknown Fearmon,U,NaN,NaN,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tom,Okker,19440222.0,178.0,Q455411,Unknown,Fearmon,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,Armistead Neely,R,NaN,USA,21.3,209533,NaN,NaN,Harry Sheridan,U,NaN,IRL,NaN,6-2 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Armistead,Neely,19470319.0,NaN,Q18085277,Harry,Sheridan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188156,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,4,209362,NaN,NaN,Aristotelis Thanos,R,NaN,GRE,20.8,209080,NaN,NaN,Blaise Bicknell,R,188.0,USA,20.2,6-4 6-4,3,RR,115.0,0.0,4.0,70.0,48.0,35.0,12.0,10.0,5.0,5.0,1.0,3.0,96.0,68.0,42.0,12.0,10.0,11.0,13.0,1103.0,9.0,1130.0,8.0,Aristotelis,Thanos,20010407.0,NaN,NaN,Blaise,Bicknell,20011126.0,188.0,Q96678039
188157,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,5,202065,NaN,NaN,Petros Tsitsipas,U,185.0,GRE,21.5,202120,NaN,NaN,Rowland Phillips,R,NaN,JAM,28.0,4-6 6-4 6-4,3,RR,161.0,7.0,2.0,94.0,55.0,37.0,23.0,15.0,9.0,12.0,1.0,1.0,94.0,56.0,40.0,20.0,15.0,4.0,8.0,808.0,23.0,1390.0,4.0,Petros,Tsitsipas,20000727.0,185.0,Q47508196,Rowland,Phillips,19940123.0,NaN,Q88392524
188158,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,1,138846,NaN,NaN,Hong Kit Jack Wong,R,NaN,HKG,23.4,105278,NaN,NaN,Alexis Klegou,U,NaN,BEN,33.0,1-6 6-4 4-1 RET,3,RR,250.0,4.0,8.0,56.0,33.0,20.0,13.0,11.0,3.0,7.0,0.0,3.0,72.0,54.0,29.0,8.0,11.0,6.0,10.0,1059.0,10.0,1881.0,1.0,Hong Kit Jack,Wong,19980830.0,NaN,Q28861638,Alexis,Klegou,19890125.0,NaN,Q33190369
188159,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,2,209409,NaN,NaN,Coleman Wong,R,NaN,HKG,17.7,200583,NaN,NaN,Delmas Ntcha,R,NaN,BEN,21.7,6-4 6-4,3,RR,98.0,2.0,2.0,68.0,38.0,24.0,18.0,10.0,6.0,8.0,1.0,5.0,58.0,39.0,24.0,7.0,10.0,5.0,9.0,NaN,NaN,NaN,NaN,Coleman,Wong,20040606.0,NaN,Q108498952,Delmas,Ntcha,20000616.0,NaN,NaN


<br>

### Tabelle mit Siegen und Niederlagen der Deutschen Nation pro Turnier

In [13]:
#Nur Zeilen, bei denen Deutschland als Gewinner hervorgeht
df_GER_win = df_matches_players[df_matches_players.winner_ioc == "GER"].groupby("tourney_name").size().reset_index(name="Siege")

#Nur Zeilen, bei denen Deutschland als Verlierer hervorgeht
df_GER_lose = df_matches_players[df_matches_players.loser_ioc == "GER"].groupby("tourney_name").size().reset_index(name="Niederlagen")

#Tabellen joinen
df_merge = pd.merge(df_GER_lose, df_GER_win, how="outer", on= "tourney_name")

#Null-Werte entfernen und Datentyp zu int64 ändern
fillna_0_astype_int64(df_merge)

#Spalte Siegesquote hinzufügen
siegesquote_funktion(df_merge)

#Spaltenreihenfolge anpassen
df_merge = df_merge[[ "tourney_name", "Siege", "Niederlagen", "Siegesquote"]]

df_merge

,tourney_name,Siege,Niederlagen,Siegesquote
0,Acapulco,26,23,0.530612
1,Adelaide,68,73,0.482270
2,Adelaide 1,0,1,0.000000
3,Adelaide 2,0,1,0.000000
4,Adelaide-2,5,4,0.555556
...,...,...,...,...
477,Davis Cup WG SF: FRG vs TCH,4,0,1.000000
478,Davis Cup WG SF: FRG vs YUG,4,0,1.000000
479,Davis Cup WG SF: SWE vs GER,4,0,1.000000
480,Davis Cup WG2 PO: LAT vs EGY,1,0,1.000000


### Top 5 Turniere, bei denen die Deutschen am schlechtesten abgeschnitten haben

In [8]:
df_filtered = df_merge.sort_values(by=["Siegesquote", "Niederlagen"], ascending=[True, False])
df_filtered = df_filtered[0:5]

# Neue Spalte erstellen
df_filtered['Land'] = ""

# Jedem Turnier das Länderkürzel zuweisen
df_filtered['Land'] = ["FRA", "ITA", "DEU", "DEU", "AUT"]
df_filtered

,tourney_id,tourney_name,Siege,Niederlagen,Siegesquote,Land
39993,2003-D007,Davis Cup WG R1: ARG vs GER,0,8,0.0,FRA
39997,2013-D006,Davis Cup WG R1: ARG vs GER,0,8,0.0,ITA
78906,1969-3936,Melbourne,0,5,0.0,DEU
78929,1970-3936,Melbourne,0,5,0.0,DEU
78966,1971-3936,Melbourne,0,5,0.0,AUT


In [9]:

fig = px.choropleth(df_filtered, locations=df_filtered["Land"], color="Siegesquote", range_color=[0,1], 
                    title="Länder der Turniere, in denen Deutschland schlecht abgeschnitten hat.")
fig.show()

Man kann erkennen, dass Deutschland hauptsächlich im europäischen Raum schlecht abschneidet.

<br>

### Top 5 Turniere, bei denen die Deutschen am besten abgeschnitten haben

In [10]:
df_filtered2 = df_merge.sort_values(by=["Siegesquote", "Siege"], ascending=[False, False])
df_filtered2 = df_filtered2[0:5]

# Neue Spalte erstellen
df_filtered2['Land'] = ""

# Jedem Turnier das Land zuweisen
df_filtered2['Land'] = ["GBR", "ARG", "USA", "CAN", "AUS"]
df_filtered2

,tourney_id,tourney_name,Siege,Niederlagen,Siegesquote,Land
166036,1974-D020,Davis Cup EUR PR: FRG vs DEN,8,0,1.0,GBR
166040,1976-D021,Davis Cup EUR PR: FRG vs DEN,8,0,1.0,ARG
166047,1975-D031,Davis Cup EUR PR: FRG vs SUI,4,0,1.0,USA
166051,1978-D028,Davis Cup EUR PR: SUI vs FRG,4,0,1.0,CAN
166055,1983-D012,Davis Cup EUR QF: BEL vs FRG,4,0,1.0,AUS


In [11]:
fig = px.choropleth(df_filtered2, locations=df_filtered2["Land"], color="Siegesquote", range_color=[0,1], title="Länder der Turniere, in denen Deutschland gut abgeschnitten hat.")
fig.show()

Man kann erkennen, dass Deutschland vorallem auf Turnieren im englischsprachigen Raum gut abschneidet.